In [1]:
from transformers import AutoModel, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("Tanrei/GPTSAN-japanese")
model = AutoModel.from_pretrained("Tanrei/GPTSAN-japanese").cuda()
x_tok = tokenizer("は、", prefix_text="織田信長", return_tensors="pt")
torch.manual_seed(0)
gen_tok = model.generate(x_tok.input_ids.cuda(), token_type_ids=x_tok.token_type_ids.cuda(), max_new_tokens=20)
tokenizer.decode(gen_tok[0])

c:\Users\Andrew\Envs\blip2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Andrew\Envs\blip2\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Andrew\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.m

'織田信長は、2004年に『戦国BASARA』のために、豊臣秀吉'

In [1]:
from lavis.models import model_zoo

print(model_zoo)

c:\Users\Andrew\Envs\blip2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Architectures                  Types
alpro_retrieval                msrvtt, didemo
blip_caption                   base_coco, large_coco
blip_classification            base
blip_feature_extractor         base
blip_image_text_matching       base, large
blip_nlvr                      nlvr
blip_pretrain                  base
blip_retrieval                 coco, flickr
blip_vqa                       vqav2, okvqa, aokvqa
blip2_Japanese_feature_extractor pretrain, pretrain_vitL, coco
blip2_Japanese                 pretrain, pretrain_vitL, coco
blip2_image_text_matching      pretrain, pretrain_vitL, coco
clip_feature_extractor         ViT-B-32, ViT-B-16, ViT-L-14, ViT-L-14-336, RN50
clip                           ViT-B-32, ViT-B-16, ViT-L-14, ViT-L-14-336, RN50


In [2]:
from lavis.models import load_model_and_preprocess

model, visual_encoder, text_encoder = load_model_and_preprocess('blip2_Japanese', 'pretrain')

In [39]:
visual_encoder

{'train': <lavis.processors.blip_processors.BlipImageTrainProcessor at 0x2b1fb01f0a0>,
 'eval': <lavis.processors.blip_processors.BlipImageEvalProcessor at 0x2b1fb01f130>}

In [ ]:
    def __getitem__(self, index):
        ann = self.annotation[index]

        image_path = os.path.join(self.vis_root, ann["image"])
        image = Image.open(image_path).convert("RGB")

        image = self.vis_processor(image)

        img_id = ann["image"].split("/")[-1].strip(".jpg").split("_")[-1]

        return {
            "image": image,
            "image_id": img_id,
            "instance_id": ann["instance_id"],
        }


In [47]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os

class SampleDataset(Dataset):

    def __init__(self, img_root, visual_processor) -> None:
        super().__init__()
        self.img_root = img_root
        self.imgs = os.listdir(img_root)
        self.visual_processor = visual_processor

    def __len__(self):
        return len(self.imgs)
    
    def __getitem__(self, index):
        image_path = os.path.join(self.img_root, self.imgs[index])
        raw_image = Image.open(image_path).convert("RGB")
        display(raw_image)
        image = self.visual_processor(raw_image)

        return { "image" : image }
    
sample_dataloader = DataLoader(SampleDataset('./samples/', visual_encoder['train']), batch_size= 8)



In [48]:
for item, raw in sample_dataloader:
    print(item['image'].shape)
    result = model.generate(item, use_nucleus_sampling=True)
    break


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'PIL.Image.Image'>

In [44]:
result

['ブロッコリー や パン が 入っ た お 弁当',
 '芝生 の 上 に 、 二 頭 の キリン が 立っ て いる',
 '白い 花瓶 に 色とりどり の 花 が 生け て ある',
 'シマウマ が 、 地面 の 草 を 食べ て いる',
 '花 柄 の 日傘 を さし た 水着 姿 の 女性',
 '乗馬 の 練習 を し て いる ところ で ある',
 '森 の 中 で 人 を 乗せ た 象 が 二 頭 歩い て いる',
 '街 中 の 時計 が 6 時 25 分 を 示し て いる']